In [1]:
import os, sys, random
import ast
import numpy as np
import pandas as pd
from typing import Tuple
from pandarallel import pandarallel
from pprint import pprint
import json
import requests
from collections import Counter

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image

# NLP
from bs4 import BeautifulSoup
import re, string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

#
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import corpus2dense
from gensim.matutils import Sparse2Corpus
from gensim.models import LdaModel, CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim import similarities
from gensim.models.ldamulticore import LdaMulticore

#
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import make_scorer, PredictionErrorDisplay, r2_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


print('\nPython version ' + sys.version)
print('pyLDAvis version ' + pyLDAvis.__version__)

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)

#
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import infer_signature, ModelSignature #, Schema, ParamSchema
from mlflow.types import Schema, ParamSchema, ParamSpec, ColSpec

# os.environ['MLFLOW_TRACKING_URI'] = './'

# ! REQUIRES CONSOLE COMMAND : mlflow ui
# depuis dossier notebooks
# at least once, to creat mlruns folder

# Utilisable seulement en local...
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



Python version 3.11.5 (main, Sep 11 2023, 13:23:44) [GCC 11.2.0]
pyLDAvis version 3.4.0

Number of CPU cores: 8
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def create_mlflow_experiment(
    experiment_name: str, artifact_location: str, tags: dict[str, str]
) -> str:
    """
    Create a new mlflow experiment with the given name and artifact location.

    Parameters:
    ----------
    experiment_name: str
        The name of the experiment to create.
    artifact_location: str
        The artifact location of the experiment to create.
    tags: dict[str,Any]
        The tags of the experiment to create.

    Returns:
    -------
    experiment_id: str
        The id of the created experiment.
    """
    try:
        experiment_id = mlflow.create_experiment(
            name=experiment_name, artifact_location=artifact_location, tags=tags
        )
    except:
        print(f"Experiment {experiment_name} already exists.")
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    mlflow.set_experiment(experiment_name=experiment_name)

    return experiment_id


def get_mlflow_experiment(
    experiment_id: str = None, experiment_name: str = None
) -> mlflow.entities.Experiment:
    """
    Retrieve the mlflow experiment with the given id or name.

    Parameters:
    ----------
    experiment_id: str
        The id of the experiment to retrieve.
    experiment_name: str
        The name of the experiment to retrieve.

    Returns:
    -------
    experiment: mlflow.entities.Experiment
        The mlflow experiment with the given id or name.
    """
    if experiment_id is not None:
        experiment = mlflow.get_experiment(experiment_id)
    elif experiment_name is not None:
        experiment = mlflow.get_experiment_by_name(experiment_name)
    else:
        raise ValueError("Either experiment_id or experiment_name must be provided.")

    return experiment


def turn_str_back_into_list(df):
    """Correct the type change due to .csv export"""

    df['title_nltk'] = df['title_nltk'].apply(ast.literal_eval)
    df['body_nltk'] = df['body_nltk'].apply(ast.literal_eval)
    df['title_spacy'] = df['title_spacy'].apply(ast.literal_eval)
    df['body_spacy'] = df['body_spacy'].apply(ast.literal_eval)
    df['all_tags'] = df['all_tags'].apply(ast.literal_eval)


def token_list_into_bow(X):
    documents = X.tolist()
    # print(documents)
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Convert Gensim corpus to dense matrix
    bow_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T

    return bow_matrix


In [3]:
# Again, this needs mlfow ui console command first -> unusable on remote server
all_experiments = client.search_experiments()
pprint(all_experiments)

train = pd.read_csv('./../data/cleaned_data/train_bow_uniques.csv', sep=',')
test = pd.read_csv('./../data/cleaned_data/test_bow_uniques.csv', sep=',')

turn_str_back_into_list(train)
turn_str_back_into_list(test)


[<Experiment: artifact_location='/home/ubuntu/Bureau/OC/Projet5_oudelet_kevin/notebooks/artifacts', creation_time=1705840547540, experiment_id='627378971600475090', last_update_time=1705840547540, lifecycle_stage='active', name='knn_optimisation', tags={'feature': 'title', 'modele': 'knn', 'nlp': 'nltk'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1705504459415, experiment_id='0', last_update_time=1705504459415, lifecycle_stage='active', name='Default', tags={}>]


In [ ]:
# add scorers : precision, r2, jaccard?


In [4]:
# test sans le preprocessor
# obligé de transformer la target mm si on ne s'sn sert pas vraiment, car grid_search.fit()
# n'accepte que des valeurs numériques.
# du coup on peut utiliser des metriques classiques pour le score (ici r2),
# mais ca n'a aucun sens metier interpretable

# ici convertir les tags en bag of words ou les one hot encoder revient exactement au meme, donc
# autant utiliser le bow, on a deja le transformer.

# ca prend trop de ressources ! Il est tps d'utiliser les nested runs de mlflow

def pipe_knn(train_df=train, feature='title_nltk', target='all_tags', test_df=test, input=['']):
    # Load your training data and labels
    X_train = train_df[feature].values
    y_train = train_df[target].values

    X_bow_matrix = token_list_into_bow(X_train)
    y_bow_matrix = token_list_into_bow(y_train)

    # Create a KNN Regressor
    knn_regressor = KNeighborsRegressor(metric=metric)

    # Create a pipeline with preprocessing and a knn regressor, to simplify gridsearch
    pipe = Pipeline(steps=[
        ("knn_regressor", knn_regressor)
    ])

    # Define hyperparameters and their possible values for grid search
    param_grid = {
        'knn_regressor__n_neighbors': [1],
        'knn_regressor__weights': ['uniform'] # , 'distance'
    }

    # Create the GridSearchCV object with multiple scoring metrics
    # scoring = {'neg_mean_squared_error': 'neg_mean_squared_error', 'r2': 'r2'}
    grid_search = GridSearchCV(pipe, param_grid=param_grid,
                            scoring='r2', cv=5, verbose=1) # add, refit='precision' for multiple scoring

    # Fit the GridSearchCV object to your training data to perform hyperparameter tuning
    grid_search.fit(X_bow_matrix, y_bow_matrix)

    # Access the best hyperparameters
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)

    # Create the KNN regressor with the best hyperparameters
    best_knn_regressor = KNeighborsRegressor(# metric=metric,
                                             n_neighbors=best_params['knn_regressor__n_neighbors'],
                                             weights=best_params['knn_regressor__weights'])

    # Create a pipeline with the preprocessor and the tuned knn regressor
    pipeline_with_tuned_knn = Pipeline(steps=[
        ("knn_regressor", best_knn_regressor)  # Use the tuned neighbor and weight values here
    ])

    # Perform cross-validation (on training set) and display the scores for each split
    # scoring = ['r2', 'neg_mean_squared_error']
    cv_scores = cross_validate(pipeline_with_tuned_knn, X_bow_matrix, y_bow_matrix, cv=5, scoring='r2')
    # print("Cross-Validation Scores (training):", '\n', cv_scores)
    print("Cross-Validation Scores:")
    pprint(cv_scores)
    for i, score in enumerate(cv_scores['test_score']):
        print(f"Split {i+1} : precision = {score}")

    return best_knn_regressor.predict(input)


# pipe_knn()


In [5]:
experiment_id = create_mlflow_experiment(
    experiment_name="knn_optimisation",
    artifact_location="./artifacts",
    tags={"modele": "knn", "feature": "title", 'nlp': 'nltk'},
)


Experiment knn_optimisation already exists.


In [11]:
experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


# define a custom model
class MyKnn(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input, params=None):
        return self.make_prediction(model_input, params)

    def make_prediction(self, model_input, params=None):
        # do something with the model input
        return pipe_knn(input=model_input)


with mlflow.start_run(run_name="testing", experiment_id=experiment_id) as run:

    # log model using autolog
    # mlflow.autolog()
    # mlflow.sklearn.autolog()
    pipe_knn(train_df=train[:100])

    # log model
    mlflow.sklearn.log_model(sk_model=MyKnn(), artifact_path="knn")
    # mlflow.pyfunc.log_model(artifact_path="knn", python_model=KNeighborsRegressor())

    # print run info
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    # print("lifecycle_stage: {}".format(run.info.lifecycle_stage)) # deprecated, use alias or tags


# J'esperais qu'mlflow allait nous permettre de contourner le probleme de
# l'entrainement du modele, qui demande bcp d'espace memoire.
# probleme : mm sans l'ui, le tracking/logging mlflow consomment enormement !
# la solution a l'air pire que le probleme...


Name: knn_optimisation
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Hyperparameters: {'knn_regressor__n_neighbors': 1, 'knn_regressor__weights': 'uniform'}
Cross-Validation Scores:
{'fit_time': array([0.00050712, 0.00046039, 0.00036788, 0.00030231, 0.0004909 ]),
 'score_time': array([0.01880622, 0.00252485, 0.00210047, 0.00295734, 0.00212431]),
 'test_score': array([0.67516111, 0.60668551, 0.64869993, 0.63764831, 0.6433043 ])}
Split 1 : precision = 0.675161110761773
Split 2 : precision = 0.6066855122136453
Split 3 : precision = 0.6486999294162054
Split 4 : precision = 0.6376483056687834
Split 5 : precision = 0.6433043046297113
run_id: 73a25a353cb348d88acf3cef6bcb903c
experiment_id: 627378971600475090
Artifact Location: /home/ubuntu/Bureau/OC/Projet5_oudelet_kevin/notebooks/artifacts
status: RUNNING
start_time: 1705847188766
end_time: None
